In [1]:
# merge all labels_nms

In [1]:
from pathlib import Path
import json

dataset_dir = Path("/mnt/ssd2/xin/repo/DART/Liebherr_Product")

label_dir_gen = dataset_dir / "labels_gen"
label_dir_orig = dataset_dir / "labels"
label_dir_background = dataset_dir / "labels_background"
label_dir_all = dataset_dir / "labels_all"
label_dir_all.mkdir(exist_ok=True)

with open(label_dir_gen / "labels_nms.json", "r") as f:
    labels_nms_gen = json.load(f)
with open(label_dir_orig / "labels_nms.json", "r") as f:
    labels_nms_orig = json.load(f)
with open(label_dir_background / "labels_nms.json", "r") as f:
    labels_nms_background = json.load(f)

with open(label_dir_gen / "stats_summary_nms.json", "r") as f:
    stats_summary_nms_gen = json.load(f)
with open(label_dir_orig / "stats_summary.json", "r") as f:
    stats_summary_nms_orig = json.load(f)["nms"]
with open(label_dir_background / "stats_summary.json", "r") as f:
    stats_summary_nms_background = json.load(f)["nms"]

with open(label_dir_gen / "stats_obj_nms.json", "r") as f:
    stats_obj_nms_gen = json.load(f)
with open(label_dir_orig / "stats_obj.json", "r") as f:
    stats_obj_nms_orig = json.load(f)["nms"]
with open(label_dir_background / "stats_obj.json", "r") as f:
    stats_obj_nms_background = json.load(f)["nms"]

with open(label_dir_gen / "no_ann.json", "r") as f:
    no_ann_gen = json.load(f)
with open(label_dir_orig / "no_ann.json", "r") as f:
    no_ann_orig = json.load(f)
with open(label_dir_background / "no_ann.json", "r") as f:
    no_ann_background = json.load(f)

In [2]:
def merge_stats_dicts(tbm):
    return {
        "avg_score": sum([x["avg_score"] * x["num_ann"] for x in tbm])
        / sum([x["num_ann"] for x in tbm]),
        "num_ann": sum([x["num_ann"] for x in tbm]),
        "num_img": sum([x["num_img"] for x in tbm]),
    }

In [3]:
# merge labels
labels_nms_all = labels_nms_orig | labels_nms_gen | labels_nms_background

# merge stats_summary
stats_summary_nms_all = merge_stats_dicts(
    [stats_summary_nms_orig, stats_summary_nms_gen, stats_summary_nms_background]
)

# merge stats_obj
tbm = [stats_obj_nms_orig, stats_obj_nms_gen, stats_obj_nms_background]
objs = stats_obj_nms_gen.keys()
stats_obj_nms_all = {obj: merge_stats_dicts([x[obj] for x in tbm]) for obj in objs}

# merge no_ann
no_ann_all = [*no_ann_orig, *no_ann_gen, *no_ann_background]

In [4]:
# save
with open(label_dir_all / "labels_nms.json", "w") as f:
    json.dump(labels_nms_all, f, indent=4)
with open(label_dir_all / "stats_summary_nms.json", "w") as f:
    json.dump(stats_summary_nms_all, f, indent=4)
with open(label_dir_all / "stats_obj_nms.json", "w") as f:
    json.dump(stats_obj_nms_all, f, indent=4)
with open(label_dir_all / "no_ann.json", "w") as f:
    json.dump(no_ann_all, f, indent=4)